# Defining multi-qubit devices, quantum circuits, and Clifford compiling
This tutorial will cover three inter-related topics:

1. `ProccesorSpec` objects, that can be used to define the "specification" of a quantum computer (e.g., device connectivity), and which are geared towards multi-qubit devices.
2. `Circuit` objects that represent quantum circuits. These are a more structured version of `Gatestring` objects, and they contain various methods that are useful for manipulating quantum circuits (e.g., simple depth compression) and interfacing `pyGSTi` with other quantum circuit standards (e.g., conversion to openqasm).
3. Clifford circuit/gate, CNOT circuit, and stabilizer state generation/measurement compilation routines.

In [1]:
from __future__ import print_function
import pygsti # the main pyGSTi module

/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Using a `ProcessorSpec` to specify a multi-qubit device.
The `ProcessorSpec` object is designed to encapsulate the specification of a quantum computer, and to hold a variety of useful things that can be derived from this information. The basic information that a `ProcessorSpec` is specified via is

1. The number of qubits in the device (and, optional, the labels of these qubits).

2. The target gate-set of the device, as either unitary matrices or using names that point to in-built unitary matrices (e.g., 'Gcnot' is a shorthand for specifying a CNOT gate). Normally will be the "primitive" gates of the device, although there it may sometimes be useful to choose other gate-sets. Currently only a discrete gate-set is supported. E.g., there is no way to specify an arbitrary $\sigma_z$-rotation as one of the gates in the device. Parameterized gates will be supported in the future.

3. The connectivity of the device.

In [2]:
# The number of qubits the device is for.
nQubits = 4

# Pick some names for the qubits. If not specified, the qubit labels default to 0, 1, 2, ...
qubit_labels = ['Q0','Q1','Q2','Q3']

# Pick a set of fundamental gates. These can be specified via in-built names,such as 'Gcnot' 
# for a CNOT gate. The full allowed set is specified in the dictionary returned by
# pygsti.tools.internalgates.get_standard_gatename_unitaries().
gate_names = ['Gi', # The idle gate
              'Gxpi2', # A X rotation by pi/2
              'Gypi2', # A Y rotation by pi/2
              'Gzpi2', # A Z rotation by pi/2
              'Gh', # The Hadamard gate
              'Gcphase']  # The controlled-Z gate.

# Additionally, we can define gates with user-specified names and actions, via a dictionary
# with keys that are strings (gate names) and values that are unitary matrices. For example,
# if you want to call the hadamard gate 'Ghad' we could do this here. The gate names should
# all start with a 'G', but are otherwise unrestricted. 
nonstd_gate_unitaries = {}

# Specify the "availability" of gates: which qubits they can be applied to. When not specified
# for a gate, it is assumed that they can be applied to all dimension-appropriate sets of qubits
# E.g., a 1-qubit gate will be assumed to be applicable to each qubit; a 2-qubit gate will be
# assumed to be applicable to all ordered pairs of qubits.
availability = {'Gcphase':[('Q0','Q1'),('Q1','Q2'),('Q2','Q3'),('Q3','Q0')]}

# Create a ProcessorSpec by handing it all of this information. This the generates a variety 
# of information about the device from this input (e.g., compilations for the Pauli operators and CNOT).
# The other defaults here will be ok for most purposes. But sometimes they will need to be changed to avoid
# slow ProcessorSpec initialization (fixes for these issues will be implemented in the future).
pspec = pygsti.obj.ProcessorSpec(nQubits, gate_names, nonstd_gate_unitaries=nonstd_gate_unitaries, 
                                 availability=availability, qubit_labels=qubit_labels)

`ProcessorSpec` objects are not particularly useful on their own. Currently, they are mostly used for interfacing with `Circuit` objects, in-built compilation algorithms, and the RB code. However, in the future we expect that they will be used for constructing circuits/gatestrings for other multi-qubit QCVV methods in `pyGSTi`.

## Defining quantum circuits with the `Circuit` object

We now introduce the `Circuit` objects. These are more a more structured version of a `GateString` object, and these objects can be easily converted between each other.
### Initializing circuits

Initializing an empty circuit is particularly simple. You just have to specify either a `line_labels` list, which is names for the wires in the circuit, or you can instead specify the number of lines as `num_lines` (the line_labels then default to integers starting at 0).

Circuits do not know what the "gates" they contain are, with one exception: they are initialized to know that a particular string corresponds to an identity/idle gate (so it will know that idle gates can be deleted and other gates commuted past this gate without changing the action of a circuit). This defaults to the 'I' string, but it can be useful to specify this as something else: often the idle identifier of a `ProcessorSpec`, as we do here.

In [3]:
#Initializing an empty circuit  over qubit 'Q0' and 'Q1'.
circuit = pygsti.obj.Circuit(line_labels=['Q0','Q1'], identity=pspec.identity)
# We can print out a circuit in a basic string format.
print(circuit)
# We can print out various properties of the circuit.
print("The circuit size is = {}".format(circuit.size()))
print("The circuit depth is = {}".format(circuit.depth()))
print("The circuit multi-qubit-gate count is = {}".format(circuit.multiQgate_count()))

Qubit Q0 -----
Qubit Q1 -----

The circuit size is = 0
The circuit depth is = 0
The circuit multi-qubit-gate count is = 0


A circuit is essentially just a load of `Label` objects, that specify what gate is applied to each wire at each step. So to specify a non-empty circuit from scratch (they can also be initialized from a GateString object) it is useful to import the `Label` object

In [4]:
from pygsti.baseobjs.label import Label as L # A shorthand for a Label

A `Label` is basically just a string, corresponding to a gate name (e.g., 'Gcnot'), and a tuple, corresponding to the qubits the gate acts on. We can initialize a label by specifying these things:

In [5]:
label_for_cnot_from_Q0_to_Q1 = L('Gcnot',('Q0','Q1'))
print(label_for_cnot_from_Q0_to_Q1.name)
print(label_for_cnot_from_Q0_to_Q1.qubits)

Gcnot
('Q0', 'Q1')


Using labels, we can initializing a non-trivial circuit. Below we create a bell-state generating circuit over 'Q0' and 'Q1'. 

There is more than one way to do this ....

In [6]:
gatestring = [L('Gh','Q0'),L('Gh','Q1'),L('Gcphase',('Q0','Q1')),L('Gh','Q0'),L('Gh','Q1')]

circuit1 = pygsti.obj.Circuit(gatestring=gatestring, line_labels=['Q0','Q1'], identity=pspec.identity)
print("- A circuit created from a gatestring *without* parallelizing:",end='\n\n')
print(circuit1)
print("The circuit size is = {}".format(circuit1.size()))
print("The circuit depth is = {}".format(circuit1.depth()))
print("The circuit multi-qubit-gate count is = {}".format(circuit1.multiQgate_count()),end='\n\n')

circuit2 = pygsti.obj.Circuit(gatestring=gatestring, line_labels=['Q0','Q1'], parallelize=True, identity=pspec.identity)
print("- A circuit created from a gatestring *with* parallelizing:",end='\n\n')
print(circuit2)
print("The circuit size is = {}".format(circuit2.size()))
print("The circuit depth is = {}".format(circuit2.depth()))
print("The circuit multi-qubit-gate count is = {}".format(circuit2.multiQgate_count()))

- A circuit created from a gatestring *without* parallelizing:

Qubit Q0 ---|Gh|-|  |-|●Q1|-|Gh|-|  |---
Qubit Q1 ---|  |-|Gh|-|●Q0|-|  |-|Gh|---

The circuit size is = 6
The circuit depth is = 5
The circuit multi-qubit-gate count is = 1

- A circuit created from a gatestring *with* parallelizing:

Qubit Q0 ---|Gh|-|●Q1|-|Gh|---
Qubit Q1 ---|Gh|-|●Q0|-|Gh|---

The circuit size is = 6
The circuit depth is = 3
The circuit multi-qubit-gate count is = 1


To define a gatestring that can be converted to a circuit in an entirely unambigious way do
.....

In [7]:
gatestring = [[L('Gh','Q0'),L('Gh','Q1')],[L('Gcphase',('Q0','Q1')),],[L('Gh','Q0'),],[L('Gh','Q1'),]]

circuit3 = pygsti.obj.Circuit(gatestring=gatestring, line_labels=['Q0','Q1'], identity=pspec.identity)
print("- A circuit created from a gatestring *with* explicit layers:",end='\n\n')
print(circuit3)
print("The circuit size is = {}".format(circuit2.size()))
print("The circuit depth is = {}".format(circuit2.depth()))
print("The circuit multi-qubit-gate count is = {}".format(circuit2.multiQgate_count()))

- A circuit created from a gatestring *with* explicit layers:

Qubit Q0 ---|Gh|-|●Q1|-|Gh|-|  |---
Qubit Q1 ---|Gh|-|●Q0|-|  |-|Gh|---

The circuit size is = 6
The circuit depth is = 3
The circuit multi-qubit-gate count is = 1


### Reading in a circuit from file.

In [8]:
nQubits = 5
# These names correspond to the 24 1-qubit Cliffords + CNOT.
gate_names = ['Gc'+format(i) for i in range(24)] + ['Gcnot']
pspec2 = pygsti.obj.ProcessorSpec(nQubits, gate_names)

In [9]:
gsList = pygsti.io.load_gatestring_list("tutorial_files/MyCircuits.txt")
line_labels = [0,1,2,3,4]
identity=pspec2.identity
circuitList = [pygsti.obj.Circuit(gatestring=gs, line_labels=line_labels, identity=identity) for gs in gsList]

In [10]:
for c in circuitList:
    print(c)

Qubit 0 -----
Qubit 1 -----
Qubit 2 -----
Qubit 3 -----
Qubit 4 -----

Qubit 0 ---|Gc11|-| ⊕1 |-|Gc12|---
Qubit 1 ---|Gc18|-| ●0 |-| ●2 |---
Qubit 2 ---|Gc12|-|Gc22|-| ⊕1 |---
Qubit 3 ---| ⊕4 |-|Gc23|-|Gc16|---
Qubit 4 ---| ●3 |-|Gc22|-|Gc21|---

Qubit 0 ---|Gc6|-|Gc4 |-|Gc5 |-|⊕1 |-|Gc15|-| ⊕1 |-|Gc13|---
Qubit 1 ---|   |-|Gc20|-|Gc23|-|●0 |-| ●2 |-| ●0 |-|Gc12|---
Qubit 2 ---|●3 |-|Gc11|-|Gc12|-|Gc9|-| ⊕1 |-|Gc12|-|Gc13|---
Qubit 3 ---|⊕2 |-|Gc13|-| ⊕4 |-|Gc2|-|Gc11|-|    |-| ⊕4 |---
Qubit 4 ---|Gc9|-|Gc17|-| ●3 |-|Gc1|-|Gc2 |-|Gc13|-| ●3 |---

Qubit 0 ---|Gc22|-|Gc13|-|Gc17|-|Gc9 |-|⊕1 |-|Gc7|-|Gc20|-|Gc21|-|Gc14|-|Gc17|-|Gc11|-|Gc13|-|Gc6 |-|Gc13|-|Gc6|-|Gc22|-|Gc1 |-|Gc8 |-|Gc5 |-|Gc19|-|Gc15|-| ⊕1 |-|Gc13|-| ⊕1 |-| ⊕1 |-|Gc7 |-| ⊕1 |-|Gc19|-|Gc19|-|Gc18|---
Qubit 1 ---|Gc12|-|Gc15|-| ●2 |-|Gc14|-|●0 |-|Gc3|-|Gc10|-| ●2 |-|    |-| ●4 |-|Gc16|-| ●2 |-| ●4 |-|Gc14|-|Gc6|-|Gc13|-| ●2 |-|Gc15|-|Gc16|-|Gc12|-| ●2 |-| ●0 |-| ●4 |-| ●0 |-| ●0 |-|Gc11|-| ●0 |-|Gc18|-| ●4 |-|Gc10|---
Qubit

### Manipulating circuits

Depth compression

In [11]:
clifford_circuit = circuitList[2]
print("The circuit *before* depth-compression using the 1-qubit gate pair-wise relations:",end='\n\n')
print(clifford_circuit)

clifford_circuit.compress_depth(oneQgate_relations=pspec2.oneQgate_relations)

print("The circuit *after* depth-compression using the 1-qubit gate pair-wise relations:",end='\n\n')
print(clifford_circuit)

The circuit *before* depth-compression using the 1-qubit gate pair-wise relations:

Qubit 0 ---|Gc6|-|Gc4 |-|Gc5 |-|⊕1 |-|Gc15|-| ⊕1 |-|Gc13|---
Qubit 1 ---|   |-|Gc20|-|Gc23|-|●0 |-| ●2 |-| ●0 |-|Gc12|---
Qubit 2 ---|●3 |-|Gc11|-|Gc12|-|Gc9|-| ⊕1 |-|Gc12|-|Gc13|---
Qubit 3 ---|⊕2 |-|Gc13|-| ⊕4 |-|Gc2|-|Gc11|-|    |-| ⊕4 |---
Qubit 4 ---|Gc9|-|Gc17|-| ●3 |-|Gc1|-|Gc2 |-|Gc13|-| ●3 |---

The circuit *after* depth-compression using the 1-qubit gate pair-wise relations:

Qubit 0 ---|    |-| ⊕1 |-|Gc15|-| ⊕1 |-|Gc13|---
Qubit 1 ---|Gc6 |-| ●0 |-| ●2 |-| ●0 |-|Gc12|---
Qubit 2 ---| ●3 |-|Gc13|-| ⊕1 |-|Gc1 |-|    |---
Qubit 3 ---| ⊕2 |-|Gc13|-| ⊕4 |-|Gc4 |-| ⊕4 |---
Qubit 4 ---|Gc20|-|    |-| ●3 |-|Gc13|-| ●3 |---



In [12]:
clifford_circuit.insert_layer([L('Gcnot',(0,1)),],1)
print(clifford_circuit)

Qubit 0 ---|    |-|●1 |-| ⊕1 |-|Gc15|-| ⊕1 |-|Gc13|---
Qubit 1 ---|Gc6 |-|⊕0 |-| ●0 |-| ●2 |-| ●0 |-|Gc12|---
Qubit 2 ---| ●3 |-|   |-|Gc13|-| ⊕1 |-|Gc1 |-|    |---
Qubit 3 ---| ⊕2 |-|   |-|Gc13|-| ⊕4 |-|Gc4 |-| ⊕4 |---
Qubit 4 ---|Gc20|-|   |-|    |-| ●3 |-|Gc13|-| ●3 |---



### Converting circuits external formats

In [13]:
openqasm = circuit2.convert_to_openqasm()
print(openqasm)

OPENQASM 2.0;
include "qelib1.inc";

qreg q[2];
creg cr[2];

h q[0];
h q[1];
barrier q[0], q[1];
cz q[0],  q[1];
barrier q[0], q[1];
h q[0];
h q[1];
barrier q[0], q[1];
measure q[0] -> cr[0];
measure q[1] -> cr[1];



There is also a `convert_to_quil()` method, which (obviously) converts to the quil format. Note that currently not all of the in-built gates can be converted to quil automatically. But, the desired gate-name conversation can be specified for both the conversion to openqasm and the conversion to quil, and a qubit-labelling conversion can also be specified.

### Simulating circuits

In [14]:
#pspec2.models['clifford'].gates

In [15]:
clifford_circuit = circuitList[3]
clifford_circuit.simulate(gateset = pspec2.models['clifford'])

OutcomeLabelDict([(('00000',), 0.031249999999999976),
                  (('00001',), 0.031249999999999976),
                  (('00010',), 0.031249999999999976),
                  (('00011',), 0.031249999999999976),
                  (('00100',), 0.031249999999999976),
                  (('00101',), 0.031249999999999976),
                  (('00110',), 0.031249999999999976),
                  (('00111',), 0.031249999999999976),
                  (('01000',), 0.031249999999999976),
                  (('01001',), 0.031249999999999976),
                  (('01010',), 0.031249999999999976),
                  (('01011',), 0.031249999999999976),
                  (('01100',), 0.031249999999999976),
                  (('01101',), 0.031249999999999976),
                  (('01110',), 0.031249999999999976),
                  (('01111',), 0.031249999999999976),
                  (('10000',), 0.031249999999999976),
                  (('10001',), 0.031249999999999976),
                  (('10010',

## Compiling Clifford gates, CNOT circuits, and stabilizer states and measurements


### Compiling a Clifford gate/circuit
 If you have a Clifford circuit that you want to use a compilers to try and create a new shorter circuit ....

In [16]:
circuit = circuitList[3]
s, p = pygsti.symplectic_rep_of_clifford_circuit(circuit, pspec=pspec2)
print(s)
print(p)

[[1 1 0 1 0 0 0 1 1 1]
 [0 1 1 1 0 1 1 1 0 0]
 [0 1 1 1 0 0 0 0 0 1]
 [1 1 1 1 1 1 1 1 1 1]
 [0 1 1 0 0 1 0 0 0 1]
 [1 1 0 0 0 0 0 1 1 0]
 [0 1 0 1 0 1 0 0 0 1]
 [0 1 1 1 1 0 0 1 1 0]
 [1 0 0 0 1 1 1 1 1 1]
 [1 1 0 1 0 0 1 1 0 1]]
[0 0 1 2 3 2 1 2 2 2]


In [17]:
new_circuit = pygsti.algorithms.compile_clifford(s,p,pspec=pspec2,iterations=200)

In [18]:
print("Original circuit:",end='\n\n')
print(circuit)
print("Original circuit depth =", circuit.depth())
print("Original circuit size =", circuit.size())
print("Original circuit mulit-qubit-gate count =", circuit.multiQgate_count())
print("New circuit:",end='\n\n')
print(new_circuit)
print("Original circuit depth =", new_circuit.depth())
print("Original circuit size =", new_circuit.size())
print("Original circuit mulit-qubit-gate count =", new_circuit.multiQgate_count())

Original circuit:

Qubit 0 ---|Gc22|-|Gc13|-|Gc17|-|Gc9 |-|⊕1 |-|Gc7|-|Gc20|-|Gc21|-|Gc14|-|Gc17|-|Gc11|-|Gc13|-|Gc6 |-|Gc13|-|Gc6|-|Gc22|-|Gc1 |-|Gc8 |-|Gc5 |-|Gc19|-|Gc15|-| ⊕1 |-|Gc13|-| ⊕1 |-| ⊕1 |-|Gc7 |-| ⊕1 |-|Gc19|-|Gc19|-|Gc18|---
Qubit 1 ---|Gc12|-|Gc15|-| ●2 |-|Gc14|-|●0 |-|Gc3|-|Gc10|-| ●2 |-|    |-| ●4 |-|Gc16|-| ●2 |-| ●4 |-|Gc14|-|Gc6|-|Gc13|-| ●2 |-|Gc15|-|Gc16|-|Gc12|-| ●2 |-| ●0 |-| ●4 |-| ●0 |-| ●0 |-|Gc11|-| ●0 |-|Gc18|-| ●4 |-|Gc10|---
Qubit 2 ---|Gc2 |-|Gc23|-| ⊕1 |-|Gc21|-|Gc3|-|Gc9|-|Gc5 |-| ⊕1 |-| ●3 |-|Gc13|-| ●3 |-| ⊕1 |-|Gc11|-| ●3 |-|Gc1|-|Gc11|-| ⊕1 |-|Gc21|-|Gc3 |-| ●3 |-| ⊕1 |-|Gc22|-|Gc4 |-|Gc12|-|Gc2 |-|Gc19|-|Gc7 |-|Gc7 |-|Gc21|-|Gc18|---
Qubit 3 ---| ⊕4 |-|Gc11|-|Gc10|-| ⊕4 |-|Gc7|-|⊕4 |-| ⊕4 |-|Gc11|-| ⊕2 |-|Gc4 |-| ⊕2 |-|Gc4 |-|Gc23|-| ⊕2 |-|⊕4 |-|Gc4 |-|Gc10|-|Gc4 |-| ⊕4 |-| ⊕2 |-|Gc16|-|Gc12|-|Gc2 |-|Gc16|-|Gc23|-| ⊕4 |-|Gc16|-|Gc19|-|Gc7 |-|Gc14|---
Qubit 4 ---| ●3 |-|Gc3 |-|Gc3 |-| ●3 |-|Gc7|-|●3 |-| ●3 |-|Gc8 |-|Gc23|-| ⊕1 |-|    |-|Gc18|-| ⊕1

In [19]:
stateprep_circuit = pygsti.algorithms.compile_stabilizer_state(s, p, pspec=pspec2, iterations=1000)

In [20]:
print(stateprep_circuit)

Qubit 0 ---|Gc2|-|⊕2 |-|Gc9 |---
Qubit 1 ---|Gc2|-|⊕4 |-|Gc14|---
Qubit 2 ---|Gc2|-|●0 |-|Gc17|---
Qubit 3 ---|Gc2|-|   |-|    |---
Qubit 4 ---|Gc2|-|●1 |-|Gc17|---



Samples a uniformly random Clifford in the "symplectic rep". This is the format that a Clifford should be provided to the Clifford compiling function in.

In [21]:
nQubits_for_clifford = 2
s, p = pygsti.random_clifford(nQubits_for_clifford)

In [22]:
# The qubits to compile the Clifford for. This doesn't need to be passed to the compiler
# if this is all the qubits in `pspec`. Nor is it necessary to specify `pspec`, ....
subsetQs = ['Q1','Q2']
circuit = pygsti.algorithms.compile_clifford(s,p,pspec=pspec,subsetQs=subsetQs)
print(circuit)

Qubit Q1 ---|Gxpi2|-|●Q2|-|Gh|-|Gypi2|-|  |---
Qubit Q2 ---| Gh  |-|●Q1|-|Gh|-|Gypi2|-|Gh|---



In [23]:
circuit = pygsti.algorithms.compile_stabilizer_state(s,p,pspec=pspec,subsetQs=subsetQs)
print(circuit)

Qubit Q1 ---|Gypi2|-|Gzpi2|-|Gh|-|●Q2|-|Gypi2|---
Qubit Q2 ---|Gypi2|-|Gzpi2|-|  |-|●Q1|-|Gypi2|---

